In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv


	1. Calculate weight for the level 12 series
	2. Use the naive logic to make forecasts for each of the level 12 series
	3. Infer forecast, ground truth values, and weights for all the higher level series by aggregating
	4. Calculalte RMSSE for all series using the equation
	5. Multiply weight by respective RMSSE and add all these products

In [3]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm

In [5]:
stv = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')

In [6]:
stv.shape

(30490, 1919)

In [10]:
stv.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [8]:
sellp = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")

In [9]:
sellp.shape

(6841121, 4)

In [11]:
sellp.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [19]:
cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")

In [20]:
cal.shape

(1969, 14)

In [21]:
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [22]:
cal["d"]=cal["d"].apply(lambda x: int(x.split("_")[1]))
sellp["id"] = sellp["item_id"] + "_" + sellp["store_id"] + "_validation"

In [23]:
# these are the last 28 days
for day in tqdm(range(1858, 1886)):
    wk_id = list(cal[cal["d"]==day]["wm_yr_wk"])[0]
    # use the week id to find the price for that week
    wk_price_df = sellp[sellp["wm_yr_wk"]==wk_id]
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
    # dollar sales per day would be the selling price times the no of units sold
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:11<00:00,  2.40it/s]


In [24]:
df["dollar_sales"] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1)

In [25]:
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True)

In [26]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [27]:
df.drop(columns=["dollar_sales"], inplace=True)

In [28]:
# final prediction forecasts
for d in range(1886, 1914):
    df["F_" + str(d)] = 0

## Forecasting unit sales and weights for all the higher level series by hierarchical aggregation

In [29]:
agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [30]:
# the unit sales on each day aggregated for all the products across all the states and stores, output series = 1
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,0,0,0,0,0,0,0,0,1,0.083333


In [31]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [33]:
# aggregating state wise, output series = 3
df.groupby(by=level_groupings[2]).sum()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
state_id,,,,,,,,,,,,,,,,,,,,,
CA,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,0,0,0,0,0,0,0,0,0,0
TX,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,0,0,0,0,0,0,0,0,0,0
WI,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# aggragating store wise, output series = 10
df.groupby(by=level_groupings[3]).sum()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
store_id,,,,,,,,,,,,,,,,,,,,,
CA_1,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,0,0,0,0,0,0,0,0,0,0
CA_2,3494,3046,2121,2324,1942,2288,2629,3729,2957,2218,...,0,0,0,0,0,0,0,0,0,0
CA_3,4739,4827,3785,4232,3817,4369,4703,5456,5581,4912,...,0,0,0,0,0,0,0,0,0,0
CA_4,1625,1777,1386,1440,1536,1389,1469,1988,1818,1535,...,0,0,0,0,0,0,0,0,0,0
TX_1,2556,2687,1822,2258,1694,2734,1691,2820,2887,2174,...,0,0,0,0,0,0,0,0,0,0
TX_2,3852,3937,2731,2954,2492,3439,2588,3772,3657,2932,...,0,0,0,0,0,0,0,0,0,0
TX_3,3030,3006,2225,2169,1726,2833,1947,2848,2832,2213,...,0,0,0,0,0,0,0,0,0,0
WI_1,2704,2194,1562,1251,2,2049,2815,3248,1674,1355,...,0,0,0,0,0,0,0,0,0,0
WI_2,2256,1922,2018,2522,1175,2244,2232,2643,2140,1836,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# aggragating category wise, output series = 3
df.groupby(by=level_groupings[4]).sum()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
cat_id,,,,,,,,,,,,,,,,,,,,,
FOODS,23178,22758,17174,18878,14603,22093,20490,27751,24862,18901,...,0,0,0,0,0,0,0,0,0,0
HOBBIES,3764,3357,2682,2669,1814,3220,2944,3986,2899,2615,...,0,0,0,0,0,0,0,0,0,0
HOUSEHOLD,5689,5634,3927,3865,2729,3898,4576,6195,4975,4056,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df.groupby(by=level_groupings[11]).sum()

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  F_1904  F_1905  F_1906  F_1907  F_1908  F_1909  \
item_id         state_id  ...                                                   
FOODS_1_001     CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   
FOODS_1_002     CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_515 TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   
HOUSEHOLD_2_516 CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   

                          F_1910  F_1911  F_1912  F_1913  
item_id         state_id                                  
FOODS_1_001     CA             0       0       0       0  
                TX             0       0       0       0  
                WI             0       0       0       0  
FOODS_1_002     CA             0       0       0       0  
                TX             0       0       0       0  
...                          ...     ...     ...     ...  
HOUSEHOLD_2_515 TX             0       0       0       0  
                WI             0       0       0       0  
HOUSEHOLD_2_516 CA             0       0       0       0  
                TX             0       0       0       0  
                WI             0       0       0       0  

[9147 rows x 1942 columns]

In [36]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df["level"] = level
    temp_df["weight"] /= 12
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████| 10/10 [00:07<00:00,  1.25it/s]


In [37]:
df["weight"] /= 12

In [39]:
print(df.shape[0], agg_df.shape[0])
print(df.shape[0] + agg_df.shape[0])

30490 12350
42840


In [40]:
# we could see the summation of all the weight values is close to 1
agg_df["weight"].sum() + df["weight"].sum()

0.9999999999999996

## Calculating RMSSE for all series using the given equation

In [41]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [42]:
# simply except last 28 days ie till day 1886
train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]
forecast_cols = [c for c in df.columns if c.find("F_") == 0]

In [43]:
df["rmsse"] = rmsse(np.array(df[ground_truth_cols]), 
                   np.array(df[forecast_cols]), np.array(df[train_series_cols]))
agg_df["rmsse"] = rmsse(np.array(agg_df[ground_truth_cols]), 
                   np.array(agg_df[forecast_cols]), np.array(agg_df[train_series_cols]))

In [44]:
# to find weighted rmse, simply multiply rmse by weight
df["wrmsse"] = df["weight"] * df["rmsse"]
agg_df["wrmsse"] = agg_df["weight"] * agg_df["rmsse"]

In [47]:
df["wrmsse"].sum() + agg_df["wrmsse"].sum()

5.35979273095658